<a href="https://colab.research.google.com/github/Tati-AG/TP1_Mineracao_de_Dados/blob/main/tratamento_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

drive.mount ('/content/drive')

Mounted at /content/drive


In [9]:
from collections import defaultdict
import pandas as pd
import numpy as np

tsv_file = '/content/drive/MyDrive/imdb_data/title.basics.tsv'

data_types = defaultdict (lambda: str, {5:'Int64', 6:'Int64'})

title_basics = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N',
                           converters = {7: lambda x: int(x) if x.isdigit() else pd.NA})

<ipython-input-9-9a0b73e55551>:9: ParserWarning: Both a converter and dtype were specified for column runtimeMinutes - only the converter will be used.
  title_basics = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N',
<ipython-input-9-9a0b73e55551>:9: ParserWarning: Both a converter and dtype were specified for column runtimeMinutes - only the converter will be used.
  title_basics = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N',
<ipython-input-9-9a0b73e55551>:9: ParserWarning: Both a converter and dtype were specified for column runtimeMinutes - only the converter will be used.
  title_basics = pd.read_table (tsv_file, sep = '\t', header = 0, dtype = data_types, na_values = '\\N',


In [15]:
movie_basics = title_basics.loc[np.where ((title_basics['titleType'].values == 'movie'))]

In [32]:
genres_by_movie = movie_basics['genres'].str.split(',').explode()

In [40]:
movie_genres = pd.DataFrame ({'genre' : genres_by_movie.unique()})

In [35]:
movie_genre_bridge = pd.DataFrame ({'movie_id' : genres_by_movie.index, 'genre' : genres_by_movie.values})

In [36]:
movie_basics.drop('genres', axis='columns', inplace=True)

In [41]:
movie_basics.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_basics.csv', index=False)
movie_genres.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_gernes.csv', index=False)
movie_genre_bridge.to_csv('/content/drive/My Drive/imdb_data/processed_data/movie_genre_bridge.csv', index=False)